## Inventory dynamics

Code from Sargent and Stachursky (2023) - https://github.com/QuantEcon/book-dp1

In [ ]:
using Pkg
Pkg.activate("..") ## because we have environment files in the parent directory
Pkg.instantiate() ## to download all missing packages


In [ ]:
# load some packages we will need today
using Distributions, QuantEcon, IterTools, Plots

In [ ]:

function create_inventory_model(; S=100, # Order size
    s=10, # Order threshold
    p=0.4) # Demand parameter
    ϕ = Geometric(p)
    h(x, d) = max(x - d, 0) + S * (x <= s)
    return (; S, s, ϕ, h)
    end

In [ ]:
function sim_inventories(model; ts_length=200)
    (; S, s, ϕ, h) = model
    X = Vector{Int32}(undef, ts_length)
    X[1] = S # Initial condition
    for t in 1:(ts_length-1)
    X[t+1] = h(X[t], rand(ϕ))
    end
    return X
    end

In [ ]:
function compute_mc(model; d_max=100)
    (; S, s, ϕ, h) = model
    n = S + s + 1 # Size of state space
    state_vals = collect(0:(S + s))
    P = Matrix{Float64}(undef, n, n)
    for (i, j) in product(1:n, 1:n)
    P[i, j] = sum((h(i-1, d) == j-1) * pdf(ϕ, d) for d in 0:d_max)
    end
    return MarkovChain(P, state_vals)
    end

In [ ]:

function compute_stationary_dist(model)
mc = compute_mc(model)
return mc.state_values, stationary_distributions(mc)[1]
end

In [ ]:
model = create_inventory_model()
sample_path = sim_inventories(model; ts_length=200)
MChain = compute_mc(model)

ψ = compute_stationary_dist(model)


In [ ]:
plot(sample_path, label=false, legend=:topleft, xlabel="time", ylabel="inventory", title="Inventory Time Series")

In [ ]:
plot(ψ, label=false, legend=:topleft, xlabel="inventory", ylabel="probability", title="Stationary Distribution of Inventory Model")

In [ ]:

X = simulate(MChain, 100000, init = 1);
plot(X, seriestype=:histogram, normed=true, label=false, legend=:topleft, xlabel="inventory", ylabel="probability", title="Histogram of Simulated Time Series")

In [ ]:

plot(X[1:100], label=false, legend=:topleft, xlabel="time", ylabel="inventory", title="Simulated Time Series")